<a href="https://colab.research.google.com/github/restrepo/anomaly/blob/main/Type_II_Dirac_Seesaw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solutions for effective Dirac neutrino masses from an Abelian Gauge Symmetry with massive chiral singlet fermions
For details see [arXiv:2108.05907](https://arxiv.org/pdf/2108.05907), Sec. 3.2

## Data Scheme
![img](https://raw.githubusercontent.com/restrepo/anomaly/main/schema.svg)

In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
pd.set_option('display.max_colwidth',500)

### Load full solutions

In [3]:
ds=pd.read_json('https://github.com/restrepo/anomaly/raw/main/solutions.json.gz')
ds.shape

(390074, 5)

## Build new `n=10,11,12` solutions

In [4]:
z=pd.Series(dtype=float)
for i in range(5,9):
    for j in range(5,9):
        if i+j>12 or i>j:
            continue
        print(i,j)
        x=ds[ds['n']==i]['solution'].reset_index(drop=True)
        y=ds[ds['n']==j]['solution'].reset_index(drop=True)
        for k in y.index:
            z=z.append(x.apply( lambda l: l+y.loc[k] ).apply(lambda l: sorted(l,key=abs))).reset_index(drop=True)

5 5
5 6
5 7
6 6


In [5]:
df=pd.DataFrame()
df['solution']=z

In [6]:
df=df[~df['solution'].astype(str).duplicated()].reset_index(drop=True)

In [7]:
df['n']=df['solution'].apply(lambda l: len(l))

In [8]:
ds=ds.append(df).reset_index(drop=True)

In [9]:
for k in ['l','k']:
    ds[k]=ds[k].apply(lambda f: [] if isinstance(f,float) else f)

In [10]:
ds=ds[~ds['solution'].astype(str).duplicated()].reset_index(drop=True)

In [11]:
ds.shape

(413572, 5)

$n=11,12$

In [12]:
z=pd.DataFrame()
for n in [5,6]:
    for m in range(1,31):
        ll=[-m,-m,-m,m,m,m]
        zz=ds[ds['n']==n].reset_index(drop=True)
        zz['solution']=zz['solution'].apply(lambda l:l+ll if m not in l and -m not in l else []).apply(
            lambda l:sorted(l,key=abs))
        zz['n']=zz['solution'].apply(len)
        z=z.append(zz).reset_index(drop=True)
        z=z[z['n']>0].reset_index(drop=True)

In [13]:
z.shape

(3766, 5)

In [14]:
ds=ds.append(z).reset_index(drop=True)

n=9,10,11,12

In [15]:
z=pd.DataFrame()

for n in [5,6,7,8]:
    for i in ds[ds['n']==n].reset_index(drop=True).index:

        zz=ds[ds['n']==n].reset_index(drop=True)
        print(f'{n}→{i}/{zz.shape[0]}',end='\r')        
        l=zz.loc[i,'solution']
        ms=[[-m,-m,m,m] for m in l if l.count(m)==1 ]

        zz=zz.loc[np.ones(len(ms)).astype(int)*i].reset_index(drop=True)

        zz['id']=zz.index

        zz['solution']=zz.apply(lambda row: row['solution']+ms[row['id']],axis='columns'
                        ).apply(lambda l:sorted(l,key=abs))
        zz=zz.drop(['id'],axis='columns')
        zz['n']=zz['solution'].apply(len)
        z=z.append(zz).reset_index(drop=True)
    print()

5→11/12
6→140/141
7→760/761
8→5568/5569


In [16]:
z.shape

(43187, 5)

In [17]:
ds=ds.append(z).reset_index(drop=True)

n=8,9,10,11,12

In [ ]:
z=pd.DataFrame()

for n in [5,6,7,8,9,10]:
    zz=ds[ds['n']==n]
    zzz=zz[zz['solution'].apply(lambda l: ( len(l)-len(set(l)) )>0 )].reset_index(drop=True)
    for i in zzz.index:
        zz=ds[ds['n']==n]
        zz=zz[zz['solution'].apply(lambda l: ( len(l)-len(set(l)) )>0 )].reset_index(drop=True)
        print(f'{n}→{i}/{zz.shape[0]}',end='\r')        
        l=zz.loc[i,'solution']
        ms=[[-m,m] for m in set(l) if l.count(m)>=2 ]

        zz=zz.loc[np.ones(len(ms)).astype(int)*i].reset_index(drop=True)

        zz['id']=zz.index
        zz['solution']=zz.apply(lambda row: row['solution']+ms[row['id']],axis='columns'
                        ).apply(lambda l:sorted(l,key=abs))
        zz=zz.drop(['id'],axis='columns')
        zz['n']=zz['solution'].apply(len)
        z=z.append(zz).reset_index(drop=True)
    print()        


6→37/38
7→210/211
8→2689/2690
9→14073/14074


In [ ]:
z.shape

In [ ]:
z[~z['solution'].astype(str).duplicated()].shape

In [ ]:
ds=ds.append(z).reset_index(drop=True)
ds.shape

In [ ]:
ds=ds[~ds['solution'].astype(str).duplicated()].reset_index(drop=True)
ds.shape

## Filter two set of repeated charges
with one of them repeated 3 times
$$(\nu,\nu[,\nu],\psi_1,\psi_2,\cdots)$$


In [ ]:
#at least one set of repeated numbers
dsp=ds[ds['solution'].apply(lambda l: len(l)-len(set(l))>0)].reset_index(drop=True)
dsp.shape

Some reordering

In [ ]:
dsp['nmax']=dsp['solution'].apply(lambda l: map(abs,l)).apply(max)
dsp=dsp.sort_values(['n','nmax']).reset_index(drop=True)
dsp2=dsp.copy()
dsp3=dsp.copy()
dsp4=dsp.copy()

#Show the first and last one solutions
#dsp#.iloc[[0,1,2,-1]]

## 1. Check effective conditions

$$ \nu+m+\delta s=0$$

$$ \psi_i+\psi_j=|s|$$

In [ ]:
def get_massless(l,s,rank=True):
    if len(l)==0:
        return []
    elif len(l)==1:
        mssls=[(l[0],l[0])]
    elif len(l)==2:
        mssls=[tuple(l)]
    else:
        mssls=set([x for x in itertools.permutations(l,2) if x[0]<=x[1] ])
    xs=[]
    massless=[] # zero eigenvalues when rank is not enough
    for x in mssls:
        if abs(x[0]+x[1])==abs(s):
            xs.append(x)  # ←→ xs=[ x for x in mssls if abs(x[0]+x[1])==abs(s) ] for rank=False
            #Check the matrix rank and store zero eigenvalues
            nr=l.count(x[0])-l.count(x[1])            
            if rank and nr>0 and abs(2*x[0])!=abs(s):
                massless=massless+ [ x[0] for i in range(nr)]
            elif rank and nr<0 and abs(2*x[1])!=abs(s):
                massless=massless+ [ x[1] for i in range(abs(nr))]
            #Check repetead massive

    #flatten list of lists into list
    massive=[x for sublist in xs for x in sublist]
    extra_massless=[]
    if rank:
        extra_massless=list(set([x for x in massive if massive.count(x)>1]))
        extra_massless=[x for x in extra_massless if abs(2*x)!=abs(s)]

    return list( set(l).difference(set(massive)) ) + massless + extra_massless  

def get_sp(massless):
    """
    For a list of numbers `x_i` find the best `sp` value such that
    all the elements of the list satisfy:
       |sp|=|x_i + x_j|
    * If the list is already empty return None
    * If not found `sp` returns 0
    """
    if len(massless)==0:
        return None
    elif len(massless)==1:
        drs=[(massless[0],massless[0])]
    elif len(massless)==2:
        drs=[tuple(massless)]
    else:
        drs=[x for x in itertools.permutations(massless,2) if x[0]<=x[1]]
    sps=set(abs(sum(x)) for x in drs)    
    for sp in sps:
        msv =([dr for dr in drs if abs(sum(dr))==sp])
        massive=set([x for sublist in msv for x in sublist])
        msl=([dr for dr in drs if abs(sum(dr))!=sp])
        massless=set( [x for sublist in msl for x in sublist])
        majorana=massless.difference(massive)
        majorana=[m for m in majorana if abs(2*m)!=sp]
        if not majorana:
            return sp
        else:
            return 0        

In [ ]:
#l=[1,-9,-9,-9,13,18,18,18,-19,-22]
#def conditions(l):
def condition(l,δ=1,rank=True):
#if True:   
    sls=[]
    ms=set([x for x in l if l.count(x)==3]+[0])
    vectorlike=[x for x in ms if l.count(-x)!=0]
    if vectorlike:
        ms=set(vectorlike)    
    νs=set([x for x in l if l.count(x)==2 or l.count(x)==3])
    #                                         exclude vector-like
    mνs=[(m,ν) for m in ms for ν in νs if m!=ν and l.count(-ν)==0]
    sold=0
    symmetry_old='XD'    
    for mν in mνs:
        m=mν[0]
        #Because of set always start at m=0 → symmetry='D'
        if m==0:
            symmetry='D'
        else:
            symmetry='X'         
        ν=mν[1]
        s=(-m-ν)/δ
        if float(s)-int(s)==0:
            s=int(s)
        else:
            continue
        #Avoid repeated solutions
        if abs(s)==abs(sold) and symmetry==symmetry_old:
            continue
        else:
            sold=s
            if m==0:
                symmetry_old='D'
            else:
                symmetry_old='X'            
        ψs=[x for x in l if x!=ν and x!=m]
        msslss=get_massless(ψs,s,rank=rank)
        sp=get_sp(msslss)
        d={'m':m,'ν':ν,'s':s,'massless':msslss,'sp':sp,'δ':δ}
        sls.append(d)
    #sp=None    
    solutions=[d for d in sls if d.get('massless')==[]]
    if not solutions:
        solutions=[d for d in sls if d.get('sp')]    
    return solutions

l=[1, 2, -6, -6, -6, 8, 9, 9, -11]
assert condition(l)[0].get('massless')==[]
l=[1, 1, 1, -4, -4, 5]
assert len(condition(l))==3
l=[1,-9,-9,-9,13,18,18,18,-19,-22]
assert set([d.get('m') for d in condition(l)])=={0,18}
l=[1, -2, -3, 5, 5, -6]
assert condition(l)[0].get('massless')==[]
#zero eigenvalues with repeated charges → -20
l=[13, 13, -16, -16, -20, -20, 23, 29, 29, 29, -32, -32]
assert condition(l,rank=False)[0]['massless']==[]
assert [-20] in [d.get('massless') for d in condition(l,rank=True)]
#zero eigenvalues with non-repeated charges → -11,-17
l=[4, -7, -7, 10, -11, 14, 15, -17, 18, -21, -22, 24]
assert condition(l,rank=False)[0]['massless']==[]
assert len(condition(l,rank=True)[0]['massless'])==2
l=[5, -5, -5, 5, 5, -7, -11, 15, 17, -20, -20, 21]
assert condition(l,δ=2)==[]

In [ ]:
def unconditional_stability(row,ZN={6:(2,3),10:(2,5),15:(3,5)},DMs={},check_massless=True):
    '''
    According to appendix of arXiv:1911.05515, 
    generalized to any N
    
    '''
    if check_massless:
        sd=[d for d in row['effective'] if d.get('massless')==[]]
    else:
        sd=row['effective']
        
    for i in range(len(sd)):
        DMs={}
        d=sd[i]
        N=abs(d['s'])
        l=[n for n in row['solution'] if n not in [d['m'],d['ν']] ]
        #Check for non-hidden chiral fermions
        if 0 in np.array(l)%N:
            DMs={'None':tuple([])}
            continue
        DM=[x for x in itertools.permutations(l,2) 
               if x[0]<=x[1] and abs(sum(x))==N ]
        DM=DM+[(n,n) for n in l if abs(2*n)==N]
        #rank 0
        for dm in DM:
            if dm[0]!=dm[1] and l.count(dm[0])!=l.count(dm[1]) and abs(2*dm[0])!=N and abs(2*dm[0])!=N:
                DMs={'None':tuple([])}
        #missing massles → TODO: Check BUG
        if check_massless and np.setdiff1d(l,[item for sublist in DM for item in sublist]).size>0:
            DMs={'None':tuple([])}
        if not DMs.get('None')==():
            try:
                p,q=ZN[N]
            except KeyError:
                DMs={'None':tuple([])}
                continue
            ps=[n*p for n in range(1,N) if n*p<N]
            qs=[n*q for n in range(1,N) if n*q<N]
            ω=[ sorted([y%N for y in x]) for x in DM]
            DMF=['ψ' if np.setdiff1d(x,ps).size==0 else 'χ' 
                   if np.setdiff1d(x,qs).size==0 else 'None' for x in  ω ]
            DMs=dict(zip(DMF,DM))
            break
    return DMs

In [ ]:
dsp['effective']=dsp['solution'].apply(condition)

In [ ]:
dsp=dsp[dsp['effective'].apply(len)>0].reset_index(drop=True)

In [ ]:
p=np.array( [2,3,5,7,11,13,17,19,23,29,31] )
q=p.copy()
zpq= [ pp*qq for pp in p for qq in q if pp<qq] 
Zpq=sorted( set( zpq ) )

In [ ]:
pqs=[ (pp,qq) for pp in p for qq in q if pp<qq]
ZN=dict(zip(Zpq,pqs))

In [ ]:
Zpq[:3]

In [ ]:
dsp['DMs']=dsp.apply(lambda row: unconditional_stability(row,ZN,check_massless=False)  ,axis='columns')

In [ ]:
dsp.shape

In [ ]:
dspm=dsp[dsp['effective'].apply(lambda l: [d for d in l if not d.get('massless')]).apply(len)>0].reset_index(drop=True)
dspm.shape

In [ ]:
#dspm[dspm['DMs'].apply(lambda d: 'ψ' in d.keys() and 'χ' in d.keys())].reset_index(drop=True)

$\delta=2$

In [ ]:
dsp2.shape

In [ ]:
dsp2['effective']=dsp2['solution'].apply(lambda l:condition(l,δ=2))

In [ ]:
dsp2=dsp2[dsp2['effective'].apply(len)>0].reset_index(drop=True)

In [ ]:
dsp2['DMs']=dsp2.apply(lambda row: unconditional_stability(row,ZN,check_massless=False)  ,axis='columns')

In [ ]:
dsp2.shape

In [ ]:
dspm2=dsp2[dsp2['effective'].apply(lambda l: [d for d in l if not d.get('massless')]).apply(len)>0].reset_index(drop=True)
dspm2.shape

In [ ]:
#dspm2[dspm2['DMs'].apply(lambda d: 'ψ' in d.keys() and 'χ' in d.keys())].reset_index(drop=True)

Massive solutions with a single $S$

In [ ]:
dslt=dspm.append(dspm2).reset_index(drop=True)
dslt.shape

In [ ]:
dslt=dslt.sort_values(['n','nmax']).reset_index(drop=True)

There are 1110 solutions most of them with $N=10$ integers

In [ ]:
dslt[dslt['n']==10].shape

In [ ]:
def count_dm(row):
#if True:
    e=row['effective']
    dm=[]
    for d in e:
        l=list(set(row['solution'].copy()))
        dmd={}
        if len(d.get('massless'))>0:
            print('NOT IMPLEMEMTED YET')
            continue
        if d.get('m'):
            l.remove( d.get('m') )
        l.remove(d.get('ν'))
        if len(l)==0:
            mssls=[()]
        elif len(l)==1:
            mssls=[(l[0],l[0])]
        elif len(l)==2:
            mssls=[tuple(l)]
        else:
            mssls=list(set([x for x in itertools.permutations(l,2) if x[0]<=x[1] ]))
        massive=[ x for x in mssls if len(mssls[0])>0 and abs(x[0]+x[1])==abs(d.get('s')) ]
        #Check generations
        gnrt=[row['solution'].count(x[0])-1 for x in massive]
        dgnrt=0
        if gnrt:
            dgnrt=max(gnrt)#int( sum( [x for x in gnrt if x>0]  ) )
        nd=len(massive)+dgnrt
        massive=[x for x in l if abs(2*x)==abs(d.get('s')) ]
        gnrt=[row['solution'].count(x)-1 for x in massive]
        mgnrt=0
        if gnrt:
            mgnrt=max(gnrt)#int( sum( [x for x in gnrt if x>0]  ) )        
        nm=len(massive)+mgnrt        
        ac=False
        if d.get('m'):
            ac=True
        uc=False
        if 'ψ' in row['DMs'].keys() and 'χ' in row['DMs'].keys():
            uc=True
        dm.append({'active':ac,'dirac':nd,'majorana':nm,'extra_generations':(dgnrt,mgnrt),'n':row['n'],'unconditional':uc,'δ':d.get('δ')})
        if len(dm)>1:
            dm=[eval(y) for y in set( [str(x) for x in dm]) ]
    
    return dm

ll=[2, 2, 3, 4, 4, -5, -6, -6, -7, 9]
row={}
d={'m': 0, 'ν': 2, 's': -2, 'massless': [], 'sp': None, 'δ': 1}
row['solution']=ll
row['effective']=[d]
row['DMs']={}
row['n']=10
assert [d.get('dirac') for d in count_dm(row)]==[4]

ll=[2, 2, 4, 4, -7, -7, -9, -9, 10, 10]
row={}
d={'m': 0, 'ν': 10, 's': -5, 'massless': [], 'sp': None, 'δ': 2}
row['solution']=ll
row['effective']=[d]
row['DMs']={}
row['n']=10
assert [d.get('dirac') for d in count_dm(row)]==[3]

In [ ]:
dslt['features']=dslt.apply(count_dm,axis='columns')

In [ ]:
fnl=dslt[~dslt['features'].astype(str).duplicated()].reset_index(drop=True)

Families of massive solutions

In [ ]:
fnl.shape

In [ ]:
fnl

__Active symmetry with D-5__


From 2 to 3 massive Dirac fermions and until a Majorana massive fermion. In the case of solutions with uncoditional stability for two dark matter candidates, if the solution includes a Majorana massive fermion, it is protected by one of the renmant symmetries. In this case the number of singlet chiral fermions is $n-3=6,8$

In [ ]:
#fnl[fnl['features'].apply(lambda l: True in [ True for d in l if d.get('active') and d.get('δ')==1 ])].reset_index(drop=True)

__Dark symmetry with D-6__

From 2 to 4 massive Dirac fermions and until a Majorana massive fermion. In the case of solutions with uncoditional stability for two dark matter candidates, if the solution includes a Majorana massive fermion

In [ ]:
#fnl[fnl['features'].apply(lambda l: True in [ True for d in l if not d.get('active') and d.get('δ')==2 ])].reset_index(drop=True)

__Active symmetry with D-6__

In [ ]:
#kk=fnl[fnl['features'].apply(lambda l: True in [ True for d in l if d.get('active') and d.get('δ')==2 ])].reset_index(drop=True)

In [ ]:
%pylab inline
df=pd.DataFrame(fnl['features'].str[0].to_list())

In [ ]:
#https://stackoverflow.com/a/34880501
#Dark → δ=1
from matplotlib.ticker import MaxNLocator
#ax = plt.figure(figsize=(4,3)).gca()
ax = plt.figure(figsize=(9,4)).gca()
ax.plot(df[ ( (df['active']==False) & (df['δ']==1) ) ]['majorana'],
         df[ ( (df['active']==False) & (df['δ']==1) ) ]['dirac'],'mo',markersize=30,label=r'$U(1)_D\ \rm{and}\ \delta=1$')
#Active → δ=1
ax.plot(df[ ( (df['active']==True) & (df['δ']==1) ) ]['majorana'],
         df[ ( (df['active']==True) & (df['δ']==1) ) ]['dirac'],'yo',markersize=25,label=r'$U(1)_X\ \rm{and}\ \delta=1$')
#Dark → δ=2
ax.plot(df[ ( (df['active']==False) & (df['δ']==2) ) ]['majorana'],
         df[ ( (df['active']==False) & (df['δ']==2) ) ]['dirac'],'go',markersize=20,label=r'$U(1)_D\ \rm{and}\ \delta=2$')
#Active → δ=2
ax.plot(df[ ( (df['active']==True) & (df['δ']==2) ) ]['majorana'],
         df[ ( (df['active']==True) & (df['δ']==2) ) ]['dirac'],'co',markersize=15,label=r'$U(1)_X\ \rm{and}\ \delta=2$')


#Dark → δ=1 → unconditional
ax.plot(df[ ( (df['active']==False) & (df['δ']==1) & (df['unconditional']==True) ) ]['majorana'],
         df[ ( (df['active']==False) & (df['δ']==1) & (df['unconditional']==True) ) ]['dirac'],'bo',
         markersize=10,label=r'$U(1)_D \to Z_p\otimes Z_q\ \rm{and}\ \delta=1$')
#Active → δ=1 → unconditional
ax.plot(df[ ( (df['active']==True) & (df['δ']==1) & (df['unconditional']==True) ) ]['majorana'],
         df[ ( (df['active']==True) & (df['δ']==1) & (df['unconditional']==True) ) ]['dirac'],'ro',
        markersize=5,label=r'$U(1)_X \to Z_p\otimes Z_q\ \rm{and}\ \delta=1$')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

ax.set_xlabel('Number of Majorana fermions',size=15)
ax.set_ylabel('Number of Dirac fermions',size=15)
ax.set_title('Massive fermions',fontsize=15)

ax.set_ylim(0.7,4.3)
ax.set_xlim(-0.3,4.3)
ax.grid()
ax.legend(loc=(1.1,0.2),fontsize=15)#,bbox_to_anchor=(2, 0.5))
plt.tight_layout()#rect=(0,0,1.6,1.2))
plt.subplots_adjust(right=0.5)
plt.savefig('number.pdf')

In [ ]:
def get_generations(row):
    l=list(row['extra_generations'])
    if l[0]:
        l[0]=l[0]+1
    if l[0]==0 and row['dirac']:
        l[0]=l[0]+1
    if l[1]:
        l[1]=l[1]+1
    if l[1]==0 and row['majorana']:
        l[1]=l[1]+1
    return tuple(l)

In [ ]:
df['generations']=df.apply(get_generations,axis='columns')

In [ ]:
#https://stackoverflow.com/a/34880501
#Dark → δ=1
from matplotlib.ticker import MaxNLocator
ax = plt.figure(figsize=(9,4)).gca()
ax.plot(df[ ( (df['active']==False) & (df['δ']==1) ) ]['generations'].str[1],
         df[ ( (df['active']==False) & (df['δ']==1) ) ]['generations'].str[0],'mo',markersize=30,label=r'$U(1)_D\ \rm{and}\ \delta=1$')
#Active → δ=1
ax.plot(df[ ( (df['active']==True) & (df['δ']==1) ) ]['generations'].str[1],
         df[ ( (df['active']==True) & (df['δ']==1) ) ]['generations'].str[0],'yo',markersize=25,label=r'$U(1)_X\ \rm{and}\ \delta=1$')
#Dark → δ=2
ax.plot(df[ ( (df['active']==False) & (df['δ']==2) ) ]['generations'].str[1],
         df[ ( (df['active']==False) & (df['δ']==2) ) ]['generations'].str[0],'go',markersize=20,label=r'$U(1)_D\ \rm{and}\ \delta=2$')
#Active → δ=2
ax.plot(df[ ( (df['active']==True) & (df['δ']==2) ) ]['generations'].str[1],
         df[ ( (df['active']==True) & (df['δ']==2) ) ]['generations'].str[0],'co',markersize=15,label=r'$U(1)_X\ \rm{and}\ \delta=2$')


#Dark → δ=1 → unconditional
ax.plot(df[ ( (df['active']==False) & (df['δ']==1) & (df['unconditional']==True) ) ]['generations'].str[1],
         df[ ( (df['active']==False) & (df['δ']==1) & (df['unconditional']==True) ) ]['generations'].str[0],'bo',
         markersize=10,label=r'$U(1)_D \to Z_p\otimes Z_q\ \rm{and}\ \delta=1$')
#Active → δ=1 → unconditional
ax.plot(df[ ( (df['active']==True) & (df['δ']==1) & (df['unconditional']==True) ) ]['generations'].str[1],
         df[ ( (df['active']==True) & (df['δ']==1) & (df['unconditional']==True) ) ]['generations'].str[0],'ro',
        markersize=5,label=r'$U(1)_X \to Z_p\otimes Z_q\ \rm{and}\ \delta=1$')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

ax.set_xlabel('Generations of Majorana fermions',size=15)
ax.set_ylabel('Generations of Dirac fermions',size=15)
ax.set_title('Massive fermions',fontsize=15)

ax.set_ylim(0.7,3.3)
ax.set_xlim(-0.3,4.3)
ax.grid()
ax.legend(loc=(1.1,0.2),fontsize=15)
plt.tight_layout()#rect=(0,0,1.6,1.2))
plt.subplots_adjust(right=0.5)
plt.savefig('generation.pdf')

In [ ]:
#https://stackoverflow.com/a/23358722/2268280
h=dslt.groupby('n')['nmax'].count()
hh=dslt[dslt.effective.str[0].str['δ']==2].groupby('n')['nmax'].count()
plt.grid(zorder=0)
plt.bar(list(h.keys()),h.values,log=True,zorder=2,label=r'$\delta=1$')
plt.bar(list(hh.keys()),hh.values,log=True,zorder=2,label=r'$\delta=2$')

plt.title('Solutions without massless fermions',fontsize=15)
plt.xlabel('$N$ [Number of integers in solutions]',size=15)
plt.ylabel('Number of solutions',size=15)
plt.legend(loc='best')
plt.savefig('solutions.pdf')


In [ ]:
dsp3['effective']=dsp3['solution'].apply(lambda l:condition(l,δ=3))

dsp3=dsp3[dsp3['effective'].apply(len)>0].reset_index(drop=True)

dsp3['DMs']=dsp3.apply(lambda row: unconditional_stability(row,ZN,check_massless=False)  ,axis='columns')

dsp3.shape

In [ ]:
dsp3[dsp3['effective'].apply(lambda l: [d for d in l if not d.get('massless')]).apply(len)>0].reset_index(drop=True)

In [ ]:
dsp4['effective']=dsp4['solution'].apply(lambda l:condition(l,δ=4))

dsp4=dsp4[dsp4['effective'].apply(len)>0].reset_index(drop=True)

#dsp4['DMs']=dsp4.apply(lambda row: unconditional_stability(row,ZN,check_massless=False)  ,axis='columns')

dsp4.shape

In [ ]:
dsp4[dsp4['effective'].apply(lambda l: [d for d in l if not d.get('massless')]).apply(len)>0].reset_index(drop=True)

# LaTeX table

In [ ]:
tm=fnl.copy()
tm['uncstab']=fnl['DMs'].apply(lambda d: 'ψ' in d.keys() and 'χ' in d.keys())

In [ ]:
tm['N']=tm.apply(lambda row: row['n']-3 if row['effective'][0]['m']!=0 else row['n'],axis='columns')

In [ ]:
Zpq

In [ ]:
tm['effective']=tm.apply(lambda row: [d for d in row['effective'] if abs(d.get('s')) in Zpq ] 
            if [d for d in row['effective'] if abs(d.get('s')) in Zpq ] else row['effective'] 
         ,axis='columns')

In [ ]:
tm['m']=tm['effective'].str[0].str['m']
tm[r'\nu']=tm['effective'].str[0].str['ν']
tm['s']=tm['effective'].str[0].str['s']
tm[r'\delta']=tm['effective'].str[0].str['δ']

In [ ]:
import re
def add_boldsymbol(ss):
    if str(ss).find(r'\boldsymbol')==-1:
        return re.sub('(\-*[0-9]+)',r'\\boldsymbol{\1}',str(ss))
    else:
        return ss

In [ ]:
kk=tm[['solution','N','m',r'\nu',r'\delta','s','uncstab']].copy()

In [ ]:
for i in kk.index:
    #for s in ['DD','DM','XD','XM']:
    kk['s']=kk.apply(lambda row: add_boldsymbol(row['s']) if row['uncstab'] else row['s'],axis='columns' )    

kkk=kk.drop('uncstab',axis='columns')#[['n','l','k','solution','gcd']]#Ref','DD','DM','XD','XM']]
def f(x):
    return  r'{}'.format(str(x).replace('[','(').replace(']',')'))

kkk.to_latex('solutions.tex',index=False,formatters=dict( [(k,f) for k in kk.columns ]) ,escape=False  )

In [ ]:
cat solutions.tex